# VADER Analysis

In [1]:
# Initial imports
import os
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv


True

In [3]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/gerrithall/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")

# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)


In [5]:
# Fetch all the news about Facebook Libra
libra_headlines = newsapi.get_everything(
    q="facebook AND libra",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Facebook Libra: {libra_headlines['totalResults']}")

# Show sample article
libra_headlines["articles"][0]


Total articles about Facebook Libra: 88


{'source': {'id': 'business-insider', 'name': 'Business Insider'},
 'author': 'ilee@insider.com (Isabelle Lee)',
 'title': 'Facebook is making it easier for crypto firms to run ads on its platform in a reversal of previous policies',
 'description': "The move is the latest update to the company's crypto policies, which have oscillated as the sector has grown in the past several years.",
 'url': 'https://markets.businessinsider.com/news/currencies/facebook-crypto-advertising-policy-change-regulatory-licenses-meta-platform-2021-12',
 'urlToImage': 'https://images2.markets.businessinsider.com/618afecd23745d001825ca14?format=jpeg',
 'publishedAt': '2021-12-02T14:57:48Z',
 'content': 'Facebook Meta signJustin Sullivan/Getty Images\r\nFacebook is making it easier for cryptocurrency companies to run advertisements on its platform, reversing its previous policies as the digital asset s… [+1899 chars]'}

In [6]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for article in libra_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        libra_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
libra_df = pd.DataFrame(libra_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()


,date,text,compound,positive,negative,neutral
0,2021-12-02,Facebook Meta signJustin Sullivan/Getty Images...,0.6486,0.159,0.000,0.841
1,2021-11-30,"Marcus, who joined the Facebook parent company...",0.0000,0.000,0.000,1.000
2,2021-11-30,"David Marcus, the head of Facebook's cryptocur...",-0.5106,0.000,0.088,0.912
3,2021-12-02,With their competitive advantage in emerging t...,0.6369,0.181,0.000,0.819
4,2021-11-16,The Deputy Governor for Financial Stability of...,0.2023,0.052,0.000,0.948


In [7]:
# Get descriptive stats from the DataFrame
libra_df.describe()

,compound,positive,negative,neutral
count,88.000000,88.000000,88.000000,88.000000
mean,0.106634,0.057409,0.028648,0.913920
std,0.369636,0.071288,0.048239,0.086293
min,-0.790600,0.000000,0.000000,0.636000
25%,0.000000,0.000000,0.000000,0.858250
50%,0.000000,0.044500,0.000000,0.936000
75%,0.366350,0.089500,0.064000,1.000000
max,0.936000,0.364000,0.219000,1.000000


# Tone Analysis

In [8]:
# Initial imports
import os
import json
from pandas import json_normalize
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


In [9]:
# Get the Tone Analyzer API Key and URL

tone_api = os.getenv("IBM_APIKEY")
tone_url = os.getenv("IBM_URL")

In [10]:
# Initialize Tone Analyser Client

# Create authentication object
authenticator = IAMAuthenticator(tone_api)

# Create tone_analyzer instance
tone_analyzer = ToneAnalyzerV3(
    version="2017-09-21",
    authenticator=authenticator
)

# Set the service endpoint
tone_analyzer.set_service_url(tone_url)


## General Tone Analysis Test

In [11]:
# Define text to analyze
text = """
Alexander could tell it was going to be a terrible, horrible, no good, very bad day. 
He went to sleep with gum in his mouth and woke up with gum in his hair.
"""

# Analyze the text's tone with the 'tone()' method.
tone_analysis = tone_analyzer.tone(
    {"text": text},
    content_type="application/json",
    content_language="en",
    accept_language="en",
).get_result()

# Display tone analysis results
print(json.dumps(tone_analysis, indent=2))

{
  "document_tone": {
    "tones": [
      {
        "score": 0.505274,
        "tone_id": "sadness",
        "tone_name": "Sadness"
      },
      {
        "score": 0.904614,
        "tone_id": "tentative",
        "tone_name": "Tentative"
      }
    ]
  },
  "sentences_tone": [
    {
      "sentence_id": 0,
      "text": "Alexander could tell it was going to be a terrible, horrible, no good, very bad day.",
      "tones": [
        {
          "score": 0.654938,
          "tone_id": "sadness",
          "tone_name": "Sadness"
        },
        {
          "score": 0.836791,
          "tone_id": "tentative",
          "tone_name": "Tentative"
        }
      ]
    },
    {
      "sentence_id": 1,
      "text": "He went to sleep with gum in his mouth and woke up with gum in his hair.",
      "tones": []
    }
  ]
}


### Transform Tone Analysis json to a DataFrame

In [12]:
# Document Tones
doc_tone_df = json_normalize(data=tone_analysis["document_tone"], record_path=["tones"])
doc_tone_df

,score,tone_id,tone_name
0,0.505274,sadness,Sadness
1,0.904614,tentative,Tentative


In [13]:
# Sentences Tones
sentences_tone_df = json_normalize(
    data=tone_analysis["sentences_tone"],
    record_path=["tones"],
    meta=["sentence_id", "text"],
)
sentences_tone_df

,score,tone_id,tone_name,sentence_id,text
0,0.654938,sadness,Sadness,0,Alexander could tell it was going to be a terr...
1,0.836791,tentative,Tentative,0,Alexander could tell it was going to be a terr...


## Customer Engagement Tone Analysis

In [14]:
# Define conversational utterances
utterances = [
    {"text": "Hello, I'm having a problem with your product.", "user": "customer"},
    {"text": "OK, let me know what's going on, please.", "user": "agent"},
    {"text": "Well, nothing is working :(", "user": "customer"},
    {"text": "Sorry to hear that.", "user": "agent"},
]

# Analyze utterances using the 'tone_chat()' method
utterance_analysis = tone_analyzer.tone_chat(
    utterances=utterances, content_language="en", accept_language="en"
).get_result()
print(json.dumps(utterance_analysis, indent=2))

{
  "utterances_tone": [
    {
      "utterance_id": 0,
      "utterance_text": "Hello, I'm having a problem with your product.",
      "tones": [
        {
          "score": 0.686361,
          "tone_id": "polite",
          "tone_name": "Polite"
        }
      ]
    },
    {
      "utterance_id": 1,
      "utterance_text": "OK, let me know what's going on, please.",
      "tones": [
        {
          "score": 0.92724,
          "tone_id": "polite",
          "tone_name": "Polite"
        }
      ]
    },
    {
      "utterance_id": 2,
      "utterance_text": "Well, nothing is working :(",
      "tones": [
        {
          "score": 0.997795,
          "tone_id": "sad",
          "tone_name": "Sad"
        }
      ]
    },
    {
      "utterance_id": 3,
      "utterance_text": "Sorry to hear that.",
      "tones": [
        {
          "score": 0.730982,
          "tone_id": "polite",
          "tone_name": "Polite"
        },
        {
          "score": 0.672499,
          "to

### Transform Chat Analysis json to DataFrame

In [15]:
chat_tone_df = json_normalize(
    data=utterance_analysis["utterances_tone"],
    record_path=["tones"],
    meta=["utterance_id", "utterance_text"],
)
chat_tone_df

,score,tone_id,tone_name,utterance_id,utterance_text
0,0.686361,polite,Polite,0,"Hello, I'm having a problem with your product."
1,0.927240,polite,Polite,1,"OK, let me know what's going on, please."
2,0.997795,sad,Sad,2,"Well, nothing is working :("
3,0.730982,polite,Polite,3,Sorry to hear that.
4,0.672499,sympathetic,Sympathetic,3,Sorry to hear that.
